In [ ]:
import numpy as np
import matplotlib.cbook as cbook
import matplotlib.image as image
import matplotlib.pyplot as plt
import cv2
import os
import math
from yu_plot_axes import run_animation

def scale_image(filename_with_path, scaling_factor=1, mirror=False):
    with cbook.get_sample_data(filename_with_path) as file:
        original_image = image.imread(file)
    new_shape = (int(original_image.shape[1] / scaling_factor), 
                 int(original_image.shape[0] / scaling_factor))
    resized_image = cv2.resize(original_image, dsize=new_shape)
    if mirror:
        resized_image = resized_image[:,::-1,:]
    return resized_image

def mirror_image(img):
    return img[:,::-1,:]

def prepare_image(filename_with_path, mirror=False):
    if filename_with_path.startswith('~'):
        filename_with_path = os.path.expanduser(filename_with_path)
    with cbook.get_sample_data(filename_with_path) as file:
        prepared_image = image.imread(file)
    if mirror:
        prepared_image = mirror_image(img=prepared_image)
    return prepared_image

def show_image(ax, prepared_image, origin, scaling_factor=1, where_position=None):
    extent=[origin[0], origin[0] + prepared_image.shape[0]*scaling_factor, 
            origin[1], origin[1] + prepared_image.shape[1]*scaling_factor]
    if 'L' in where_position:
        extent[0] -= prepared_image.shape[0]*scaling_factor
        extent[1] -= prepared_image.shape[0]*scaling_factor
    ax.imshow(prepared_image, extent=extent)        

def val_or_func_to_val(val_or_func, arg):
    if isinstance(val_or_func, int) or isinstance(val_or_func, float):
        val = val_or_func
    else:
        if isinstance(val_or_func, tuple):
            val = val_or_func[0] * arg + val_or_func[1]
        else:    
            val = val_or_func(arg)
    return val

def make_images_xs_ys_scalings(images_poss_speeds, x_lims):
    images_xs_ys_scalings = []
    scaling_factor=(x_lims[1] - x_lims[0]) / 640
    y = [-20 * scaling_factor, 10 * scaling_factor]
    i = 0
    for img, pos, speed in images_poss_speeds.values():
        if speed >= 0:
            where_position="L"
            x_lims[0] = min(x_lims[0], pos - img.shape[0]*scaling_factor)
        else:
            img=mirror_image(img=img)
            where_position="R"
            x_lims[1] = max(x_lims[1], pos + img.shape[0]*scaling_factor)
        images_xs_ys_scalings.append([img, (speed, pos), y[i], scaling_factor, where_position])
        i += 1
    axes_lims = {'x' : x_lims, 'y' : [0, 80 * scaling_factor]}
    return axes_lims, images_xs_ys_scalings

def animate_images(i, ax, nb_frames, images_xs_ys_scalings, axes_lims, title_func, *args):
    ax.clear()
    for spine_id in ['right', 'top', 'left']:
        ax.spines[spine_id].set_visible(False)
    ax.set_yticks([], [])   
    ax.set_xlim(axes_lims['x'])
    ax.set_ylim(axes_lims['y'])

    xs=[]
    for img, x, y, scaling, where_position in images_xs_ys_scalings:
        x_value = val_or_func_to_val(val_or_func=x, arg=i)
        xs.append(x_value)
        y_value = val_or_func_to_val(val_or_func=y, arg=i)
        show_image(ax, img, (x_value, y_value), scaling, where_position)
    
    plot_title = title_func(i, xs, *args)
    ax.set_title(plot_title)

def show_distance(i, xs, cars_data, time_unit="h", distance_unit="km"):
    result = ""
    j=0
    for colour, car_data in cars_data.items():
        result += (colour + " car is at " + str(xs[j]) + distance_unit 
                         + ", its speed is " + str(abs(car_data[2])) + distance_unit + "/" + time_unit + ".\n")
        j += 1
    if len(xs) > 1:
        result=("The distance between the cars is " + str(abs(xs[1] - xs[0])) + distance_unit + ". \n" + result)
    return ("Time elapsed: " + str(i) + " " + time_unit + ". \n" + result)


In [ ]:

car_red_R = prepare_image('~/Documents/Python/Tutoring/car_red.png')
car_blue_R = prepare_image('~/Documents/Python/Tutoring/car_blue.png', mirror=True)

cars_data = {"Red" : [car_red_R, 0, 40],
             "Blue": [car_blue_R, 1050, -30]}
axes_lims, images_xs_ys_scalings = make_images_xs_ys_scalings(images_poss_speeds = cars_data, 
                                                              x_lims=[0, 1300])
anim = run_animation(figsize=(15, 5), nb_frames=100, 
                     func_name=animate_images, fargs=[images_xs_ys_scalings, axes_lims, show_distance, cars_data], interval=200)
anim